In [ ]:
import pandas as pd
import geopandas as gpd
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_image
from functools import partial
from rasterio.enums import MergeAlg
from shapely.geometry import LineString
import matplotlib as plt
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt2
import rasterio as rio
import numpy as np
from scipy import stats
import requests
from io import BytesIO
from zipfile import ZipFile
from functools import partial
import pygeohash as gh
from shapely.geometry import Polygon

In [ ]:
url = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/master/data/NYC_AQ.csv'
NYC_AQ = pd.read_csv(url)

url = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/master/data/nyc_polygon.geojson'
NYC_geojson = gpd.read_file(url)


In [ ]:
def remove_outliers_zscore(geodataframe, threshold=3):
    # Extracting coordinates from geometry
    coords = np.array([(point.x, point.y) for point in geodataframe.geometry])

    # Calculating z-scores for both longitude and latitude
    z_scores_lon = np.abs(stats.zscore(coords[:, 0]))
    z_scores_lat = np.abs(stats.zscore(coords[:, 1]))

    # Find outliers for both longitude and latitude
    outliers = (z_scores_lon > threshold) | (z_scores_lat > threshold)

    # Remove outliers
    cleaned_geodataframe = geodataframe[~outliers].copy()
    return cleaned_geodataframe

In [ ]:
NYC_AQ

In [ ]:
print(NYC_geojson.head())

In [ ]:
columns_to_keep = ['latitude', 'longitude', 'pm25']
NYC_AQ = NYC_AQ[columns_to_keep]

NYC_AQ = NYC_AQ[(NYC_AQ['latitude'] != 0) & (NYC_AQ['longitude'] != 0)]

NYC_AQ['longitude'] = pd.to_numeric(NYC_AQ['longitude'])
NYC_AQ['latitude'] = pd.to_numeric(NYC_AQ['latitude'])

print(NYC_AQ)

In [ ]:
geohash_precision = 7
NYC_AQ['geohash']=NYC_AQ.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=geohash_precision), axis=1)
NYC_AQ.head()

In [ ]:
avg_pm25 = NYC_AQ.groupby('geohash').agg({'pm25': 'mean'}).reset_index()
avg_pm25.rename(columns={'pm25': 'avg_pm25'}, inplace=True)
avg_pm25

In [ ]:
NYC_AQ = NYC_AQ.merge(avg_pm25, on='geohash', how='left')
NYC_AQ

In [ ]:
NYC_AQ = gpd.GeoDataFrame(NYC_AQ)
NYC_AQ.head()

In [ ]:
NYC_AQ = gpd.GeoDataFrame(NYC_AQ, geometry = gpd.points_from_xy(x=NYC_AQ['longitude'], y=NYC_AQ['latitude']))
NYC_AQ = NYC_AQ.set_crs('EPSG:4326')
NYC_AQ



In [ ]:
def decode_geohash(geohash):
    """Decode the geohash to its bounding box (longitude and latitude ranges)."""

    # Geohash character-to-binary mapping
    base32_map = '0123456789bcdefghjkmnpqrstuvwxyz'
    base32_dict = {char: "{:05b}".format(i) for i, char in enumerate(base32_map)}

    # Split geohash into bits for longitude and latitude
    bits = ''.join(base32_dict[c] for c in geohash)
    lon_bits = bits[::2]
    lat_bits = bits[1::2]

    # Function to decode bits to a range
    def decode_range(bits, range_min, range_max):
        for bit in bits:
            mid = (range_min + range_max) / 2
            if bit == '1':
                range_min = mid
            else:
                range_max = mid
        return (range_min, range_max)

    # Decode longitude and latitude ranges
    lon_range = decode_range(lon_bits, -180, 180)
    lat_range = decode_range(lat_bits, -90, 90)

    # Return the bounding box as a dictionary
    return {
        'w': lon_range[0],
        'e': lon_range[1],
        's': lat_range[0],
        'n': lat_range[1],
    }

In [ ]:
temp = pd.DataFrame(columns=['bbox'])
temp['bbox'] = NYC_AQ['geohash'].apply(decode_geohash)
# Convert bounding boxes to polygons
temp['geometry'] = temp['bbox'].apply(lambda b: Polygon([
    (b['w'], b['s']),
    (b['w'], b['n']),
    (b['e'], b['n']),
    (b['e'], b['s'])
]))
NYC_AQ = gpd.GeoDataFrame(NYC_AQ, geometry = temp['geometry'])
NYC_AQ

In [ ]:
NYC_AQ_geojson = gpd.sjoin(NYC_AQ, NYC_geojson, predicate="within")
NYC_AQ_geojson

In [ ]:
# sampled_geohash_data = NYC_AQ_geojson.groupby('geohash').apply(lambda x: x.sample(frac=0.6))
# sampled_geohash_data

In [ ]:
print(len(NYC_AQ['geohash'].unique()))

In [ ]:
print(NYC_AQ['geohash'].value_counts().head(70))


In [ ]:
pm25_avg_max = NYC_AQ['avg_pm25'].max()
pm25_avg_min = NYC_AQ['avg_pm25'].min()
print(pm25_avg_max)
print(pm25_avg_min)

In [ ]:
NYC_AQ_geojson = NYC_AQ_geojson.to_crs('EPSG:32618')

In [ ]:
unique_polygon = NYC_AQ.drop_duplicates(subset='geometry')
unique_polygon

In [ ]:
unique_lengths = unique_polygon['geohash'].astype(str).apply(len).unique()
unique_lengths

In [ ]:
unique_polygon['layer2'] = 7
unique_polygon

In [ ]:
AQ_NYC_raster = make_geocube(
        vector_data = unique_polygon,
        measurements = ['avg_pm25', 'layer2'],
        resolution = (-10, 10),
        rasterize_function=partial(rasterize_image, merge_alg=MergeAlg.add),
        fill= 0,
        output_crs="EPSG:32618")

In [ ]:
image_path = r'./output_samples/2layers.tiff'
AQ_NYC_raster.rio.to_raster(image_path)

In [ ]:
# Load the raster file
with rio.open(image_path) as src:
    data = src.read(1)  # read the first band

# Set up the figure
plt2.figure(figsize=(10, 10))

# Display the raster data with a colormap
plt2.imshow(data, cmap='viridis', vmin=pm25_avg_min, vmax=pm25_avg_max)  # set the range of the colormap
plt2.colorbar(label='PM2.5 Levels')  # add a color bar

# Add titles and labels if necessary
plt2.title('PM2.5 Distribution in NYC')
plt2.xlabel('Longitude Index')
plt2.ylabel('Latitude Index')

# Show the plot
plt2.show()

In [ ]:
# Load the raster file
with rio.open(pm25_avg_max) as src:
    data = src.read(1)  # read the first band

# Set up the figure
plt2.figure(figsize=(10, 10))

# Display the raster data with a colormap
plt2.imshow(data, cmap='viridis', vmin=2.0657861653999996, vmax=12.31804457)  # set the range of the colormap
plt2.colorbar(label='PM2.5 Levels')  # add a color bar

# Add titles and labels if necessary
plt2.title('PM2.5 Distribution in NYC')
plt2.xlabel('Longitude Index')
plt2.ylabel('Latitude Index')

# Show the plot
plt2.show()

In [ ]:
landsat = rio.open("./output_samples/AQ_NYC_raster_overlayed_pm25.tiff")

In [ ]:
# The CRS
landsat.crs

In [ ]:
# The bounds
landsat.bounds

In [ ]:
# The number of bands available
landsat.count

In [ ]:
# The band numbers that are available
landsat.indexes

In [ ]:
# Number of pixels in the x and y directions
landsat.shape

In [ ]:
# The 6 parameters that map from pixel to real space
landsat.transform